# LangGraph Neo4j MCP Agent - Diagnostic Install

This notebook installs packages **one at a time** to identify which one causes kernel crashes.

## Instructions

1. Run each cell in order
2. After each install + restart, note if the kernel crashes
3. The problematic package is the last one you installed before the crash

---

## 0. Environment Info (Run First)

Check what's already installed before making any changes.

In [ ]:
import sys
import platform

print("=" * 60)
print("ENVIRONMENT INFO")
print("=" * 60)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Machine: {platform.machine()}")
print(f"Executable: {sys.executable}")
print()

In [ ]:
# Check pre-installed packages that might conflict
import subprocess

check_packages = [
    "langchain", "langchain-core", "langchain-community",
    "langgraph", "langchain-aws", "langchain-mcp-adapters",
    "mcp", "httpx", "boto3", "botocore", "pydantic",
    "anyio", "httpcore", "nest-asyncio"
]

print("=" * 60)
print("PRE-INSTALLED PACKAGES")
print("=" * 60)

for pkg in check_packages:
    try:
        result = subprocess.run(
            [sys.executable, "-m", "pip", "show", pkg],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            for line in result.stdout.split("\n"):
                if line.startswith("Version:"):
                    print(f"{pkg:30} {line.split(':')[1].strip()}")
                    break
        else:
            print(f"{pkg:30} not installed")
    except Exception as e:
        print(f"{pkg:30} error: {e}")

In [ ]:
# Check available memory (useful for debugging OOM crashes)
try:
    import psutil
    mem = psutil.virtual_memory()
    print(f"Total memory: {mem.total / (1024**3):.1f} GB")
    print(f"Available: {mem.available / (1024**3):.1f} GB")
    print(f"Used: {mem.percent}%")
except ImportError:
    print("psutil not available - cannot check memory")
except Exception as e:
    print(f"Memory check failed: {e}")

---

## 1. Install: nest-asyncio (Safe - usually no issues)

In [ ]:
%pip install --upgrade "nest-asyncio>=1.6.0" -q
print("nest-asyncio installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
import nest_asyncio
print(f"nest_asyncio version: {nest_asyncio.__version__}")
print("STATUS: nest-asyncio OK")

---

## 2. Install: httpx

In [ ]:
%pip install --upgrade "httpx>=0.28.0" -q
print("httpx installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
import httpx
print(f"httpx version: {httpx.__version__}")
print("STATUS: httpx OK")

---

## 3. Install: boto3 (AWS SDK)

In [ ]:
%pip install --upgrade "boto3>=1.36.0" -q
print("boto3 installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
import boto3
print(f"boto3 version: {boto3.__version__}")
print("STATUS: boto3 OK")

---

## 4. Install: mcp (Model Context Protocol)

This is often where issues start - has many dependencies.

In [ ]:
%pip install --upgrade "mcp>=1.9.2" -q
print("mcp installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
try:
    import mcp
    print(f"mcp version: {mcp.__version__}")
    print("STATUS: mcp OK")
except Exception as e:
    print(f"ERROR importing mcp: {e}")

---

## 5. Install: langchain-core (Foundation)

Installing langchain-core first before the full langchain package.

In [ ]:
%pip install --upgrade "langchain-core>=0.3.0,<1.0.0" -q
print("langchain-core installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
try:
    import langchain_core
    print(f"langchain_core version: {langchain_core.__version__}")
    print("STATUS: langchain-core OK")
except Exception as e:
    print(f"ERROR importing langchain_core: {e}")

---

## 6. Install: langgraph

In [ ]:
%pip install --upgrade "langgraph>=0.2.60,<1.0.0" -q
print("langgraph installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
try:
    import langgraph
    print(f"langgraph version: {langgraph.__version__}")
    from langgraph.prebuilt import create_react_agent
    print("create_react_agent import: OK")
    print("STATUS: langgraph OK")
except Exception as e:
    print(f"ERROR: {e}")

---

## 7. Install: langchain

In [ ]:
%pip install --upgrade "langchain>=0.3.0,<1.0.0" -q
print("langchain installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
try:
    import langchain
    print(f"langchain version: {langchain.__version__}")
    print("STATUS: langchain OK")
except Exception as e:
    print(f"ERROR: {e}")

---

## 8. Install: langchain-aws

In [ ]:
%pip install --upgrade "langchain-aws>=0.2.0,<1.0.0" -q
print("langchain-aws installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
try:
    from langchain_aws import ChatBedrockConverse
    print("ChatBedrockConverse import: OK")
    print("STATUS: langchain-aws OK")
except Exception as e:
    print(f"ERROR: {e}")

---

## 9. Install: langchain-mcp-adapters

**This is often the problematic package** - it has strict version requirements.

In [ ]:
%pip install --upgrade "langchain-mcp-adapters>=0.1.0" -q
print("langchain-mcp-adapters installed. Restart kernel now.")

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
# VERIFY: Run after kernel restart
print("Kernel restarted successfully!")
try:
    from langchain_mcp_adapters.client import MultiServerMCPClient
    print("MultiServerMCPClient import: OK")
    print("STATUS: langchain-mcp-adapters OK")
except Exception as e:
    print(f"ERROR: {e}")

---

## 10. Final Verification - All Packages

In [ ]:
import importlib.metadata

packages = [
    "nest-asyncio",
    "httpx",
    "boto3",
    "mcp",
    "langchain-core",
    "langgraph",
    "langchain",
    "langchain-aws",
    "langchain-mcp-adapters",
]

print("=" * 60)
print("FINAL PACKAGE VERSIONS")
print("=" * 60)

for pkg in packages:
    try:
        version = importlib.metadata.version(pkg)
        print(f"{pkg:30} {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"{pkg:30} NOT INSTALLED")

In [ ]:
# Test all critical imports together
print("Testing all imports...")

errors = []

try:
    import nest_asyncio
    print("  nest_asyncio: OK")
except Exception as e:
    errors.append(f"nest_asyncio: {e}")

try:
    import httpx
    print("  httpx: OK")
except Exception as e:
    errors.append(f"httpx: {e}")

try:
    import boto3
    print("  boto3: OK")
except Exception as e:
    errors.append(f"boto3: {e}")

try:
    import mcp
    print("  mcp: OK")
except Exception as e:
    errors.append(f"mcp: {e}")

try:
    from langgraph.prebuilt import create_react_agent
    print("  langgraph.prebuilt.create_react_agent: OK")
except Exception as e:
    errors.append(f"langgraph: {e}")

try:
    from langchain_aws import ChatBedrockConverse
    print("  langchain_aws.ChatBedrockConverse: OK")
except Exception as e:
    errors.append(f"langchain_aws: {e}")

try:
    from langchain_mcp_adapters.client import MultiServerMCPClient
    print("  langchain_mcp_adapters.client.MultiServerMCPClient: OK")
except Exception as e:
    errors.append(f"langchain_mcp_adapters: {e}")

print()
if errors:
    print("ERRORS FOUND:")
    for e in errors:
        print(f"  - {e}")
else:
    print("ALL IMPORTS SUCCESSFUL!")

---

## Troubleshooting Tips

If kernel crashes persist, try these debugging steps:

In [ ]:
# Check for pip dependency conflicts
!pip check 2>&1 | head -50

In [ ]:
# Show dependency tree for langchain-mcp-adapters
!pip show langchain-mcp-adapters 2>/dev/null || echo "Not installed"

In [ ]:
# Verbose import to see exactly where it fails
import sys
import importlib

def verbose_import(module_name):
    """Try to import a module and show detailed error info."""
    print(f"Attempting to import: {module_name}")
    try:
        mod = importlib.import_module(module_name)
        print(f"  SUCCESS: {mod}")
        if hasattr(mod, '__version__'):
            print(f"  Version: {mod.__version__}")
        if hasattr(mod, '__file__'):
            print(f"  Location: {mod.__file__}")
        return mod
    except Exception as e:
        print(f"  FAILED: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test problematic imports
print("=" * 60)
verbose_import("langchain_mcp_adapters")
print()
verbose_import("langchain_mcp_adapters.client")

In [ ]:
# Alternative: Try installing with verbose output to see what's happening
# Uncomment and run if needed:

# !pip install langchain-mcp-adapters -v 2>&1 | tail -100

In [ ]:
# Nuclear option: Uninstall all and start fresh
# Uncomment if needed:

# !pip uninstall -y langchain langchain-core langchain-community langgraph langchain-aws langchain-mcp-adapters mcp
# print("All langchain packages removed. Restart kernel and reinstall.")

---

## Alternative: Install with Pinned Versions

If the above fails, try these **known working combinations**:

In [ ]:
# Option A: Latest 0.3.x langchain (pre-1.0, more stable)
# Uncomment to try:

# %pip install --upgrade \
#     "langchain==0.3.14" \
#     "langchain-core==0.3.28" \
#     "langgraph==0.2.60" \
#     "langchain-aws==0.2.10" \
#     "langchain-mcp-adapters==0.1.3" \
#     "mcp==1.3.0" \
#     "httpx==0.28.0" \
#     "nest-asyncio==1.6.0"

In [ ]:
# Option B: Use conda instead of pip (if available)
# Uncomment to try:

# !conda install -c conda-forge langchain langgraph -y

---

## Agent Code (Run after all installs succeed)

### Configuration

**ACTION REQUIRED**: Copy values from `.mcp-credentials.json`

In [ ]:
# =============================================================================
# REPLACE THESE VALUES with your credentials from .mcp-credentials.json
# =============================================================================

GATEWAY_URL = "YOUR_GATEWAY_URL_HERE"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN_HERE"

# AWS Bedrock settings
AWS_REGION = "us-west-2"
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"

# Validate
if "YOUR_" in GATEWAY_URL or "YOUR_" in ACCESS_TOKEN:
    print("ERROR: Replace GATEWAY_URL and ACCESS_TOKEN above!")
else:
    print(f"Gateway: {GATEWAY_URL[:60]}...")
    print(f"Token:   {ACCESS_TOKEN[:30]}...")
    print("Configuration OK!")

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

from langchain_aws import ChatBedrockConverse
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.client import MultiServerMCPClient

SYSTEM_PROMPT = """You are a helpful Neo4j database assistant. Query the database schema first, then answer questions using Cypher."""

# Global agent
agent = None

async def setup_agent():
    global agent
    print("Connecting to MCP server...")
    
    client = MultiServerMCPClient({
        "neo4j": {
            "transport": "streamable_http",
            "url": GATEWAY_URL,
            "headers": {"Authorization": f"Bearer {ACCESS_TOKEN}"},
        }
    })
    
    tools = await client.get_tools()
    print(f"Loaded {len(tools)} tools: {[t.name for t in tools]}")
    
    llm = ChatBedrockConverse(model=MODEL_ID, region_name=AWS_REGION, temperature=0)
    agent = create_react_agent(model=llm, tools=tools, prompt=SYSTEM_PROMPT)
    print("Agent ready!")
    return agent

asyncio.get_event_loop().run_until_complete(setup_agent())

In [ ]:
def query(question: str) -> str:
    """Ask the agent a question."""
    async def _ask():
        result = await agent.ainvoke({"messages": [("human", question)]})
        messages = result.get("messages", [])
        return getattr(messages[-1], "content", str(messages[-1])) if messages else "No response"
    
    print(f"Q: {question}")
    answer = asyncio.get_event_loop().run_until_complete(_ask())
    print(f"A: {answer}")
    return answer

In [ ]:
_ = query("What is the database schema?")